In [1]:
pip install deep-translator

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install textblob

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install googletrans==4.0.0-rc1

Note: you may need to restart the kernel to use updated packages.


In [4]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\asuar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\asuar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
import requests
import time
import json
from bs4 import BeautifulSoup
from datetime import datetime
from textblob import TextBlob
import pandas as pd
from nltk.tokenize import word_tokenize
from collections import Counter
from nltk.tokenize import RegexpTokenizer
from deep_translator import GoogleTranslator

In [6]:
url = 'https://www.cnbc.com/stocks/'
response = requests.get(url)

In [7]:
soup = BeautifulSoup(response.content, "html.parser")


In [8]:
# finds all of the headlines and the timestamps of when each headline was scrapped
timestamps = []
headlines = []
for headline in soup.find_all("a", {"class": "Card-title"}):
    current_time = datetime.now()
    timestamps.append(current_time)
    headlines.append(headline.text)
    time.sleep(0.01)

In [9]:
#Finds all of the urls attached to each headline and appends them to a list called urls
urls = []
for a in soup.find_all("a", {"class": "Card-title"}):
    urls.append(a['href'])
urls

['https://www.cnbc.com/2023/06/17/the-japanese-equity-rally-could-broaden-out-some-small-and-mid-cap-names-may-get-a-boost-.html',
 'https://www.cnbc.com/2023/06/16/the-ipo-market-may-put-pressure-on-the-fed-cramer-says.html',
 'https://www.cnbc.com/2023/06/16/cramer-wait-for-a-price-break-next-week-while-the-markets-overbought.html',
 'https://www.cnbc.com/2023/06/16/jim-cramer-explains-why-dominos-pizza-stock-soared-this-week.html',
 'https://www.cnbc.com/2023/06/16/cramers-lightning-round-dominion-energy-is-a-little-too-risky.html',
 'https://www.cnbc.com/2023/06/16/nvidia-gets-upgrade-amid-10percent-weekly-gain-heres-what-the-pros-say.html',
 'https://www.cnbc.com/2023/06/16/toyota-stock-best-week-since-2009.html',
 'https://www.cnbc.com/2023/06/16/these-two-cruise-stocks-and-this-chipmaker-led-the-market-higher-this-week.html',
 'https://www.cnbc.com/2023/06/16/stocks-moving-big-midday-spce-irbt-cava-sofi.html',
 'https://www.cnbc.com/2023/06/16/the-reshoring-boom-is-stoking-deman

In [10]:
#Calculates the sentiment score for each of the headlines and appends them to a list
sentiment_score = []
for headline in headlines:
    blob = TextBlob(headline)
    sentiment_score.append(blob.sentiment.polarity)
sentiment_score

[-0.125,
 0.0,
 0.0,
 0.0,
 -0.19375,
 0.0,
 1.0,
 0.25,
 0.5,
 0.0,
 0.0,
 0.5,
 0.5,
 0.0,
 -0.04999999999999999,
 0.0,
 0.0,
 0.0,
 0.0,
 -0.2,
 0.0,
 0.5,
 -0.13333333333333333,
 0.5,
 0.3,
 0.0,
 0.16666666666666666,
 0.1,
 0.0,
 0.0,
 0.5,
 0.13636363636363635,
 0.35,
 0.0,
 -0.3]

In [11]:
#Creates a dataframe with the column sentiment_score
df = pd.DataFrame({'sentiment_score':sentiment_score})
df.head()

,sentiment_score
0,-0.12500
1,0.00000
2,0.00000
3,0.00000
4,-0.19375


In [12]:
#Creates a column called 'sentiment' that takes the sentiment score and calculates the corresponding category
df.loc[df['sentiment_score'].between(-1, -0.2, inclusive=True), 'sentiment'] = 'negative'
df.loc[df['sentiment_score'].between(-0.2, 0.2, inclusive=False), 'sentiment'] = 'neutral'
df.loc[df['sentiment_score']>=0.2, 'sentiment'] = 'positive'

C:\Users\asuar\AppData\Local\Temp\ipykernel_23588\1165563063.py:2: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated in favour of `both` or `neither`.
  df.loc[df['sentiment_score'].between(-1, -0.2, inclusive=True), 'sentiment'] = 'negative'
C:\Users\asuar\AppData\Local\Temp\ipykernel_23588\1165563063.py:3: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated in favour of `both` or `neither`.
  df.loc[df['sentiment_score'].between(-0.2, 0.2, inclusive=False), 'sentiment'] = 'neutral'


In [13]:
#Creates a headline column
df['headline'] = headlines


In [14]:
#Finds the relevant words in each headline and appends them to a list
relevant_words_list = []

stopwords = nltk.corpus.stopwords.words("english")

tokenizer = RegexpTokenizer(r'\w+')

for headline in headlines:
    tokens = tokenizer.tokenize(headline)
    words = [w for w in tokens if w.lower() not in stopwords]
    relevant_words_list.append(words)

In [15]:
#Creates a column of the relevant words of each headline
df['relevant_words'] = relevant_words_list

In [16]:
#Translates the headline column to italian 
def translate_to_italian(text):
    italian_translation = GoogleTranslator(source='auto', target='it').translate(text)
    return italian_translation

In [17]:
#Translates the headline column to spanish and 
def translate_to_spanish(text):
    spanish_translation = GoogleTranslator(source='auto', target='es').translate(text)
    return spanish_translation

In [18]:
#Creates an italian translation column in the dataframe
df['headline_it'] = df['headline'].apply(translate_to_italian)

In [19]:
#Creates a spanish translation column column in the dataframe
df['headline_spanish'] = df['headline'].apply(translate_to_spanish)

In [20]:
#Rounds the sentiment score column to two decimal places
df['sentiment_score'] = df['sentiment_score'].round(2)

In [21]:
#Creates a column of the urls
df['url'] = urls

In [22]:
#Creates a column of the timestamps
df['timestamp'] = timestamps

In [23]:
df

,sentiment_score,sentiment,headline,relevant_words,headline_it,headline_spanish,url,timestamp
0,-0.12,neutral,The Japanese equity rally could broaden out. S...,"[Japanese, equity, rally, could, broaden, smal...",Il rally dell'azionario giapponese potrebbe am...,El repunte de la renta variable japonesa podrí...,https://www.cnbc.com/2023/06/17/the-japanese-e...,2023-06-17 17:02:38.023070
1,0.00,neutral,"The IPO market may put pressure on the Fed, Cr...","[IPO, market, may, put, pressure, Fed, Cramer,...",Il mercato delle IPO potrebbe esercitare press...,El mercado de OPI puede ejercer presión sobre ...,https://www.cnbc.com/2023/06/16/the-ipo-market...,2023-06-17 17:02:38.035334
2,0.00,neutral,Wait for a price break next week while the mar...,"[Wait, price, break, next, week, market, overb...",Aspetta una rottura dei prezzi la prossima set...,Espere una ruptura de precios la próxima seman...,https://www.cnbc.com/2023/06/16/cramer-wait-fo...,2023-06-17 17:02:38.048434
3,0.00,neutral,Jim Cramer explains why Domino's Pizza stock s...,"[Jim, Cramer, explains, Domino, Pizza, stock, ...",Jim Cramer spiega perché le scorte di Domino's...,Jim Cramer explica por qué las acciones de Dom...,https://www.cnbc.com/2023/06/16/jim-cramer-exp...,2023-06-17 17:02:38.060654
4,-0.19,neutral,"Cramer's Lightning Round: Dominion Energy is ""...","[Cramer, Lightning, Round, Dominion, Energy, l...","Cramer's Lightning Round: Dominion Energy è ""u...",Lightning Round de Cramer: Dominion Energy es ...,https://www.cnbc.com/2023/06/16/cramers-lightn...,2023-06-17 17:02:38.072481
5,0.00,neutral,Nvidia gets price target hike amid 10% weekly ...,"[Nvidia, gets, price, target, hike, amid, 10, ...",Nvidia ottiene un aumento del target di prezzo...,Nvidia obtiene un aumento de precio objetivo e...,https://www.cnbc.com/2023/06/16/nvidia-gets-up...,2023-06-17 17:02:38.088454
6,1.00,positive,Toyota defies skeptics as stock seals best wee...,"[Toyota, defies, skeptics, stock, seals, best,...",Toyota sfida gli scettici mentre le azioni sig...,Toyota desafía a los escépticos y las acciones...,https://www.cnbc.com/2023/06/16/toyota-stock-b...,2023-06-17 17:02:38.104404
7,0.25,positive,These two cruise stocks and this chipmaker led...,"[two, cruise, stocks, chipmaker, led, market, ...",Questi due titoli da crociera e questo produtt...,Estas dos acciones de cruceros y este fabrican...,https://www.cnbc.com/2023/06/16/these-two-crui...,2023-06-17 17:02:38.120461
8,0.50,positive,Stocks making the biggest moves midday: Virgin...,"[Stocks, making, biggest, moves, midday, Virgi...",Le azioni che fanno le mosse più importanti a ...,Acciones que realizan los mayores movimientos ...,https://www.cnbc.com/2023/06/16/stocks-moving-...,2023-06-17 17:02:38.136507
9,0.00,neutral,The reshoring boom is stoking demand for robot...,"[reshoring, boom, stoking, demand, robotics, s...",Il boom del reshoring sta alimentando la doman...,El auge de la relocalización está alimentando ...,https://www.cnbc.com/2023/06/16/the-reshoring-...,2023-06-17 17:02:38.152455


In [24]:
#Writes the dataframe to a csv file
df.to_csv('data/headlines/headlines_data.csv')

In [25]:
"""

Beginning API Dataframe 

"""

'\n\nBeginning API Dataframe \n\n'

In [26]:
"""
Apple AAPL
Microsoft MSFT
Google GOOG
Amazon AMZN
Meta META
"""

api_key = 'QGFWZRXMOEUDV9B6'


# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY&symbol=AAPL&apikey=' + api_key
r = requests.get(url)
apple_data = r.json()

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY&symbol=MSFT&apikey=' + api_key
r = requests.get(url)
microsoft_data = r.json()


# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY&symbol=GOOG&apikey=' + api_key
r = requests.get(url)
google_data = r.json()

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY&symbol=AMZN&apikey=' + api_key
r = requests.get(url)
amazon_data = r.json()

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY&symbol=META&apikey=' + api_key
r = requests.get(url)
meta_data = r.json()


In [27]:
def transform_json(json_data, symbol):
    
    # Turns the json data into a dataframe
    temp_df = pd.DataFrame.from_dict(json_data['Monthly Time Series'])
    
    #Changes the dates from column names to a single column named 'Date'
    temp_df_melted = temp_df.melt( 
        var_name="Date",
        ignore_index=False
        )
    temp_df_melted = temp_df_melted.reset_index()
    
    #Pivoting the data from a long to a wide format and renames the columns
    temp_df_pivoted = temp_df_melted.pivot(index='Date', columns='index', values='value')
    temp_df_pivoted.rename(
            columns = {"1. open":"open_price","2. high":"high_price","3. low": "low_price","4. close":"close_price", "5. volume":"volume"},
            inplace = True
        )
    temp_df_transformed = temp_df_pivoted.reset_index()
    temp_df_transformed.rename(columns ={'Date':'date'}, inplace =True)
    
    #Removing the index name from the index
    temp_df_transformed = temp_df_transformed.rename_axis(None, axis=1)
    
    #Adding a column of the stock symbol
    symbols = []
    for i in range(len(temp_df_transformed)):
        symbols.append(symbol)
    temp_df_transformed['symbol'] = symbols
    
    return temp_df_transformed

In [28]:
meta_dataframe = transform_json(meta_data,'META')

In [29]:
meta_dataframe

,date,open_price,high_price,low_price,close_price,volume,symbol
0,2012-06-29,28.8920,33.4500,25.5200,31.0950,667910500,META
1,2012-07-31,31.2500,32.8800,21.6100,21.7100,520189700,META
2,2012-08-31,21.5000,22.4500,18.0300,18.0580,1151944900,META
3,2012-09-28,18.0800,23.3700,17.5500,21.6600,1058643700,META
4,2012-10-31,22.0800,24.2500,18.8000,21.1100,1100938300,META
...,...,...,...,...,...,...,...
128,2023-02-28,148.0300,197.1600,147.0600,174.9400,772199855,META
129,2023-03-31,174.5900,212.1700,171.4300,211.9400,690450690,META
130,2023-04-28,208.8400,241.6850,207.1300,240.3200,439810216,META
131,2023-05-31,238.6150,268.6500,229.8500,264.7200,487059059,META


In [30]:
amazon_dataframe = transform_json(amazon_data, 'AMZN')

In [31]:
amazon_dataframe

,date,open_price,high_price,low_price,close_price,volume,symbol
0,1999-12-31,87.2500,113.0000,76.0000,76.1300,246476100,AMZN
1,2000-01-31,81.5000,91.5000,58.4400,64.5600,262209000,AMZN
2,2000-02-29,67.5000,85.9400,63.0600,68.8800,207457700,AMZN
3,2000-03-31,67.6300,75.2500,60.0000,67.0000,156584600,AMZN
4,2000-04-28,65.4400,68.6300,40.8100,55.1900,162012500,AMZN
...,...,...,...,...,...,...,...
278,2023-02-28,102.5300,114.0000,92.3186,94.2300,1364198308,AMZN
279,2023-03-31,93.8700,103.4900,88.1200,103.2900,1349466996,AMZN
280,2023-04-28,102.3000,110.8600,97.7050,105.4500,1222124091,AMZN
281,2023-05-31,104.9500,122.9200,101.1500,120.5800,1432007632,AMZN


In [32]:
google_dataframe = transform_json(google_data, 'GOOG')

In [33]:
google_dataframe

,date,open_price,high_price,low_price,close_price,volume,symbol
0,2014-04-30,558.7100,604.8300,502.8000,526.6600,67171200,GOOG
1,2014-05-30,527.1100,567.8400,503.3000,559.8900,36527700,GOOG
2,2014-06-30,560.7000,582.4500,538.7500,575.2800,37898400,GOOG
3,2014-07-31,578.3200,599.6500,565.0100,571.6000,34516100,GOOG
4,2014-08-29,570.4000,587.3400,560.0000,571.6000,27586000,GOOG
...,...,...,...,...,...,...,...
106,2023-02-28,99.7400,108.8200,88.8600,90.3000,754465909,GOOG
107,2023-03-31,90.1600,107.5100,89.7700,104.0000,725590594,GOOG
108,2023-04-28,102.6700,109.6300,102.3800,108.2200,461168243,GOOG
109,2023-05-31,107.7200,127.0500,104.5000,123.3700,620306000,GOOG


In [34]:
microsoft_dataframe = transform_json(microsoft_data, 'MSFT')

In [35]:
microsoft_dataframe

,date,open_price,high_price,low_price,close_price,volume,symbol
0,1999-12-31,91.0600,119.9400,90.8700,116.7500,630488900,MSFT
1,2000-01-31,117.3700,118.6200,94.8700,97.8700,637437600,MSFT
2,2000-02-29,98.5000,110.0000,88.1200,89.3700,667243800,MSFT
3,2000-03-31,89.6200,115.0000,88.9400,106.2500,1014093800,MSFT
4,2000-04-28,94.4400,96.5000,65.0000,69.7500,1129073300,MSFT
...,...,...,...,...,...,...,...
278,2023-02-28,248.0000,276.7600,245.4700,249.4200,615540778,MSFT
279,2023-03-31,250.7600,289.2700,245.6100,288.3000,747751976,MSFT
280,2023-04-28,286.5200,308.9300,275.3700,307.2600,551356990,MSFT
281,2023-05-31,306.9700,335.9400,303.4000,328.3900,600453246,MSFT


In [36]:
apple_dataframe = transform_json(apple_data, 'AAPL')

In [37]:
apple_dataframe

,date,open_price,high_price,low_price,close_price,volume,symbol
0,1999-12-31,101.0000,118.0000,91.0600,102.8100,84091200,AAPL
1,2000-01-31,104.8700,121.5000,86.5000,103.7500,112099800,AAPL
2,2000-02-29,104.0000,119.9400,97.0000,114.6200,65355200,AAPL
3,2000-03-31,118.5600,150.3800,114.0000,135.8100,77663900,AAPL
4,2000-04-28,135.5000,139.5000,104.8700,124.0600,77342900,AAPL
...,...,...,...,...,...,...,...
278,2023-02-28,143.9700,157.3800,141.3200,147.4100,1307294493,AAPL
279,2023-03-31,146.8300,165.0000,143.9000,164.9000,1520461315,AAPL
280,2023-04-28,164.2700,169.8500,159.7800,169.6800,967580718,AAPL
281,2023-05-31,169.2800,179.3500,164.3100,177.2500,1275052503,AAPL


In [38]:
stocks_df = pd.concat([meta_dataframe, amazon_dataframe,google_dataframe,microsoft_dataframe,apple_dataframe])

In [39]:
stocks_df

,date,open_price,high_price,low_price,close_price,volume,symbol
0,2012-06-29,28.8920,33.4500,25.5200,31.0950,667910500,META
1,2012-07-31,31.2500,32.8800,21.6100,21.7100,520189700,META
2,2012-08-31,21.5000,22.4500,18.0300,18.0580,1151944900,META
3,2012-09-28,18.0800,23.3700,17.5500,21.6600,1058643700,META
4,2012-10-31,22.0800,24.2500,18.8000,21.1100,1100938300,META
...,...,...,...,...,...,...,...
278,2023-02-28,143.9700,157.3800,141.3200,147.4100,1307294493,AAPL
279,2023-03-31,146.8300,165.0000,143.9000,164.9000,1520461315,AAPL
280,2023-04-28,164.2700,169.8500,159.7800,169.6800,967580718,AAPL
281,2023-05-31,169.2800,179.3500,164.3100,177.2500,1275052503,AAPL


In [40]:
stocks_df.to_csv('data/stocks/stocks_data.csv')

In [41]:
import sqlite3
import csv

In [42]:
#creates database
database = 'extended_case_etl.sqlite'
conn = sqlite3.connect(database)

#creates cursor
cur = conn.cursor()

#drops table if already exists
drop_table_query = "DROP TABLE IF EXISTS headlines"
cur.execute(drop_table_query)
conn.commit()

drop_table_query = "DROP TABLE IF EXISTS stocks"
cur.execute(drop_table_query)
conn.commit()

#creates headlines table
create_table_query = "CREATE TABLE IF NOT EXISTS headlines(\
                        sentiment_score FLOAT, sentiment VARCHAR,\
                        headline VARCHAR, relevant_words ARRAY,\
                        headline_it VARCHAR, headline_spanish VARCHAR,\
                        url VARCHAR, timestamp TIMESTAMP)"
cur.execute(create_table_query)
conn.commit()


#creates stocks table
create_table_query = "CREATE TABLE IF NOT EXISTS stocks(\
                        date DATE, open_price FLOAT,\
                        high_price FLOAT, low_price FLOAT,\
                        close_price FLOAT, volume INTEGER,\
                        symbol VARCHAR)"
cur.execute(create_table_query)
conn.commit()

# Open and read the headlines CSV file
with open('data/headlines/headlines_data.csv', 'r') as file:
    # Create a CSV reader object
    csv_reader = csv.reader(file)
    
    # Insert the data into the headlines table
    cur.executemany('INSERT INTO headlines (sentiment_score,\
                        sentiment, headline, relevant_words,\
                        headline_it, headline_spanish, url, \
                        timestamp) VALUES (?, ?, ?, ?, ?, ?, ?, ?)', (row[1:] for row in csv_reader))
conn.commit()

# Open and read the headlines CSV file
with open('data/stocks/stocks_data.csv', 'r') as file:
    # Create a CSV reader object
    csv_reader = csv.reader(file)
    
    # Skips header row 
    next(csv_reader)
    
    # Insert the data into the headlines table
    cur.executemany('INSERT INTO stocks (date, open_price,\
                        high_price, low_price,\
                        close_price, volume,\
                        symbol) VALUES (?, ?, ?, ?, ?, ?, ?)', (row[1:] for row in csv_reader))
    
conn.commit()
#conn.close()

In [44]:
cur.execute("SELECT * FROM headlines").fetchall()

[('sentiment_score',
  'sentiment',
  'headline',
  'relevant_words',
  'headline_it',
  'headline_spanish',
  'url',
  'timestamp'),
 (-0.12,
  'neutral',
  'The Japanese equity rally could broaden out. Some small- and mid-cap names may get a boost ',
  "['Japanese', 'equity', 'rally', 'could', 'broaden', 'small', 'mid', 'cap', 'names', 'may', 'get', 'boost']",
  "Il rally dell'azionario giapponese potrebbe ampliarsi. Alcuni nomi a piccola e media capitalizzazione potrebbero ricevere una spinta",
  'El repunte de la renta variable japonesa podrÃ\xada ampliarse. Algunos nombres de pequeÃ±a y mediana capitalizaciÃ³n pueden recibir un impulso',
  'https://www.cnbc.com/2023/06/17/the-japanese-equity-rally-could-broaden-out-some-small-and-mid-cap-names-may-get-a-boost-.html',
  '2023-06-17 17:02:38.023070'),
 (0.0,
  'neutral',
  'The IPO market may put pressure on the Fed, Cramer says',
  "['IPO', 'market', 'may', 'put', 'pressure', 'Fed', 'Cramer', 'says']",
  'Il mercato delle IPO potre